In [61]:
#import dependencies
import pandas as pd
import datetime
import numpy as np
import re
from nltk.corpus import wordnet

In [2]:
#read in count_df
df = pd.read_csv('/Users/gregfinin/Class/Final_Project_WSB/stocks_df_final.csv')

In [3]:
#df
df

,Ticker Name,Ticker Count
0,GME,52231
1,AMC,22200
2,NEXT,11282
3,BB,7235
4,GOOD,6508
...,...,...
6930,ZUMZ,0
6931,ZVO,0
6932,ZWRK,0
6933,ZWRKU,0


In [4]:
# get ticker names and put into list
ticker_names = df['Ticker Name']
ticker_list = ticker_names.to_list()

0         GME
1         AMC
2        NEXT
3          BB
4        GOOD
        ...  
6930     ZUMZ
6931      ZVO
6932     ZWRK
6933    ZWRKU
6934    ZWRKW
Name: Ticker Name, Length: 6935, dtype: object

In [8]:
# lists to store values in for loop
tickers_to_use = []
tickers_not_to_use = []

In [13]:
# check list of tickers against english dictionary. add non-word tickers to one list and word tickers to other list
for ticker in ticker_list:
    if not wordnet.synsets(ticker):
        tickers_to_use.append(ticker)
    else:
        tickers_not_to_use.append(ticker)

In [17]:
# Get list of non-word tickerss
combined_list = set(tickers_to_use).intersection(ticker_list)

In [18]:
# make list into dataframe.
combined_list = pd.DataFrame(combined_list)
combined_list

,0
0,SCHN
1,CRON
2,UMH
3,INVE
4,STWOU
...,...
6196,PWOD
6197,CBIO
6198,VRME
6199,MASI


In [51]:
# load in reddit data
titles_df = pd.read_csv('/Users/gregfinin/Class/Final_Project_WSB/Data/wallstreetbets_subs.csv')

In [23]:
titles_df = titles_df['title']
titles_df

0                                 Any predictions for $WLL?
1                                          In Musk We Trust
2                            How to update your BTC targets
3                 GME Short Squeeze: A WallStreetBets Movie
4                          Let’s coordinate a pump and dump
                                ...                        
717091                                        American weed
717092                                  Relief Therapeutics
717093          $QTMM The ugliest stock page I've ever seen
717094    Send Doge to the moon so I can leave my abusiv...
717095    WE Raised the Curve on Thursday, Just as Hedgi...
Name: title, Length: 717096, dtype: object

In [27]:
# create list for storing values
ticker_names_list = []
ticker_counts = []

In [33]:
# count tickers in submissions and append lists
for ticker in combined_list[0]:
    count = titles_df.str.contains(rf"\s({ticker})\s", flags=re.IGNORECASE, regex=True).sum()
    ticker_names_list.append(ticker)
    ticker_counts.append(count)

In [37]:
# create dataframe for non word tickers
non_word_ticker_df = pd.DataFrame(ticker_names_list)

In [38]:
# add counts
non_word_ticker_df['count'] = ticker_counts
non_word_ticker_df

,0,count
0,0,267
1,SCHN,1
2,CRON,7
3,UMH,0
4,INVE,0
...,...,...
6197,PWOD,0
6198,CBIO,0
6199,VRME,0
6200,MASI,0


In [41]:
#sort by values
non_word_tickers = non_word_ticker_df.sort_values(by='count', ascending=False)

In [42]:
# check df
non_word_tickers

,0,count
3607,GME,53911
1718,AMC,22993
5801,NOK,6498
5580,ELSE,3348
4145,TD,1248
...,...,...
2298,STAR.PRI,0
2297,LGVW,0
2296,RACA,0
2294,RWT,0


In [43]:
# remove tickers with no mentions
non_word_tickers = non_word_tickers.where(non_word_tickers['count'] != 0)

In [45]:
# drop nulls
final_non_word_tickers = non_word_tickers.dropna()

In [46]:
#final count of non word tickers (dont know why else made it through)
final_non_word_tickers

,0,count
3607,GME,53911.0
1718,AMC,22993.0
5801,NOK,6498.0
5580,ELSE,3348.0
4145,TD,1248.0
...,...,...
5355,SAR,1.0
497,LYG,1.0
577,KMT,1.0
6191,ZYXI,1.0


In [48]:
# save as csv
final_non_word_tickers.to_csv(path_or_buf='/Users/gregfinin/Project/non_word_tickers.csv',index=False)

In [101]:
# make dataframe for titles andd dates
title_df = pd.DataFrame({
    'title': title_by_date['title'],
    'date_created': pd.to_datetime(title_by_date['created_utc'], unit='s')
})
title_df

,title,date_created
0,Any predictions for $WLL?,2021-01-12 03:17:42
1,In Musk We Trust,2021-01-12 03:11:30
2,How to update your BTC targets,2021-01-12 03:11:22
3,GME Short Squeeze: A WallStreetBets Movie,2021-01-12 03:06:45
4,Let’s coordinate a pump and dump,2021-01-12 03:02:44
...,...,...
717091,American weed,2021-02-04 22:34:45
717092,Relief Therapeutics,2021-02-04 22:34:38
717093,$QTMM The ugliest stock page I've ever seen,2021-02-04 22:34:36
717094,Send Doge to the moon so I can leave my abusiv...,2021-02-04 22:34:33


In [102]:
# convert date created column
title_df['date'] = title_df['date_created'].dt.date

In [103]:
#drop date created
title_df.drop(columns='date_created')

,title,date
0,Any predictions for $WLL?,2021-01-12
1,In Musk We Trust,2021-01-12
2,How to update your BTC targets,2021-01-12
3,GME Short Squeeze: A WallStreetBets Movie,2021-01-12
4,Let’s coordinate a pump and dump,2021-01-12
...,...,...
717091,American weed,2021-02-04
717092,Relief Therapeutics,2021-02-04
717093,$QTMM The ugliest stock page I've ever seen,2021-02-04
717094,Send Doge to the moon so I can leave my abusiv...,2021-02-04


In [111]:
# One of the tickers was 0 as an int and was throwing off my for loop.
list_of_non_zero_tickers.remove(0)

In [75]:
list_of_non_zero_tickers = non_zero_tickers.to_list()

In [112]:
# ready to run this but it will create 1249 columns for each ticker - might want to review which ones before running
for ticker in list_of_non_zero_tickers:
    title_df[ticker] = np.where(title_df['title'].str.lower().str.contains(ticker), 1, 0)

KeyboardInterrupt: 